# 01. Import and Install Dependencies

In [1]:
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2                                # Computer Vison Library OpenCV
import numpy as np                        # numpy arrays structure our datasets
import os                                 # work with operating systems file path
from matplotlib import pyplot as plt      # matplotlib just helps us visualize images a little bit easier
import mediapipe as mp                    # mediapipe (https://google.github.io/mediapipe/)

# 02. Keypoints Using MP FashMesh

In [3]:
mp_face_mesh = mp.solutions.face_mesh              # FaceMesh Model has -- 478 -- Keypoints 
mp_drawing_styles = mp.solutions.drawing_styles    # Drawing styles
mp_drawing = mp.solutions.drawing_utils            # Drawing utilites

In [4]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)   # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                     # Image is no longer writeable
    results = model.process(image)                    # Make prediction
    image.flags.writeable = True                      # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)    # COLOR CONVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image,face_landmarks):
    # Draw face connections
    mp_drawing.draw_landmarks(
            image= image,
            landmark_list= face_landmarks,
            connections= mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec= None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
    
    # Draw face countours (FACEMESH_CONTOURS = frozenset().union(*[FACEMESH_LIPS, FACEMESH_LEFT_EYE, FACEMESH_LEFT_EYEBROW, FACEME) connections
    mp_drawing.draw_landmarks(
            image= image,
            landmark_list= face_landmarks,
            connections= mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec= None,
            connection_drawing_spec= mp_drawing_styles.get_default_face_mesh_contours_style())
    
    # Draw eye iris ( FACEMESH_IRISES = frozenset().union(*[FACEMESH_LEFT_IRIS, FACEMESH_RIGHT_IRIS]) ) connections
    mp_drawing.draw_landmarks(
            image= image,
            landmark_list= face_landmarks,
            connections= mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec= None,
            connection_drawing_spec= mp_drawing_styles.get_default_face_mesh_iris_connections_style())

# 03. Extract Keypoints Values 

In [6]:
def extract_keypoints(results):
    face = np.array([[res.x , res.y , res.z] for res in results.multi_face_landmarks[0].landmark]).flatten() if results.multi_face_landmarks[0].landmark else np.zeros(478*3)
    return np.concatenate([face])

In [7]:
# for res in results.multi_face_landmarks[0].landmark]).flatten():
#    if results.multi_face_landmarks[0].landmark:
#        np.array([[res.x , res.y , res.z]
#    else :
#      np.zeros(478*3)

# 04. Setup Folder for Collection

In [8]:
DATA_PATH = os.path.join('ML_Data')         # Path for exported data, numpy arrays

actions = np.array(['true', 'lie'])       # Actions that we try to detect

no_sequences = 2                       # videos worth of data per action (True-02 vdos, Lie-02 Vdos)

sequence_length = 30              # Videos are going to be 30 frames in length

In [9]:
# Create Folders for save keypoints values

for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

#  05. Collect Keypoints Values for Training and Testing

In [10]:
for action in actions:   # Loop through actions
    
    for sequence in range(no_sequences):    # Loop through sequences (videos)
        
        VDO_PATH = 'Datasets/' + action +'/' + str(sequence) + '/' + str(sequence) + '.mp4'  # /Datasets/true/0.mp4
        cap = cv2.VideoCapture(VDO_PATH)   # Read Video
        
        with mp_face_mesh.FaceMesh(   # set mediapipe model
            max_num_faces =1,
            refine_landmarks = True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as face_mesh:
            for frame_num in range(sequence_length):
                
                ret, frame = cap.read()  # Read feed

                print("Processing Video Path:",VDO_PATH)          # Processing Video Path:Datasets/true/0/0.mp4

                image, results = mediapipe_detection(frame, face_mesh)                        # Make detections

#                 ### Draw landmarks ###
#                 cv2.waitKey(1)
#                 if results.multi_face_landmarks:
#                     for face_landmarks in results.multi_face_landmarks:
#                         draw_landmarks(image,face_landmarks)
#                         cv2.imshow('OpenCV feed', image)

                keypoints = extract_keypoints(results)                                       # Extract keypoints
            
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))     # Export keypoints
                
                print("Keypoints Shape:",keypoints.shape)                              # Keypoints Shape:(1434,)
                
                np.save(npy_path, keypoints)                                                    # Save Keypoints


            cap.release()     # Release feed
            
            cv2.waitKey(1)    # wait for 1 Secound
            
            print("____________Finish__________________")                    # ____________Finish__________________
            
#     cv2.destroyAllWindows()   # Close All OpenCV Windows

Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Path: Datasets/true/0/0.mp4
Keypoints Shape: (1434,)
Processing Video Pat

# 6. Preprocess Data and Create Labels and Features

In [11]:
from sklearn.model_selection import train_test_split # Scikit-learn for our evaluation metrics
from tensorflow.keras.utils import to_categorical   # Converting Data (Encoded)

In [12]:
label_map = {label:num for num, label in enumerate(actions)}  # Actions Label Map  {'true': 0, 'lie': 1} 

In [13]:
sequences, labels = [], []                # like : Features(X) Data , labels(Y) Data
for action in actions:              
    for sequence in range(no_sequences):  
        window = []                       # Save all of diff.frames paricular sequence
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))  # load np array
            window.append(res)                # Append all res 
        sequences.append(window)             # Append all window
        labels.append(label_map[action])   # Append all actions on labels

In [14]:
X = np.array(sequences)  # sequences
y = to_categorical(labels).astype(int)  # labels

# Train & Testing Partition
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) 

In [15]:
# X.shape : (4, 30, 1434)  -----> ( Sequences(videos) , framers per vdo, keypoints shape [478 * 3] )
# y.shape : (4, 2) -------->   ( Sequences(videos), labels[actions])

# X_train.shape : (3, 30, 1434) 
# X_test.shape : (1, 30, 1434)

# y_train.shape : (3, 2) 
# y_test.shape : (1, 2) 

# 7. Build and Train LSTM Neural Network

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [17]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [18]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1434)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [19]:
actions.shape[0]

2

In [20]:
res = [.7, 0.2, 0.1]

In [21]:
actions[np.argmax(res)]

'true'

In [22]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [23]:
model.fit(X_train, y_train, epochs=1, callbacks=[tb_callback])

1/1 [==============================] - 5s 5s/step - loss: 0.7069 - categorical_accuracy: 0.3333


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            383744    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 538,274
Trainable params: 538,274
Non-trai

# 8. Make Predictions

In [25]:
res = model.predict(X_test)

In [26]:
res

array([[0.53625345, 0.46374655]], dtype=float32)

In [27]:
np.sum(res[0])

1.0

In [28]:
actions[np.argmax(res[0])]

'true'

In [29]:
y_test

array([[1, 0]])

In [30]:
actions[np.argmax(y_test[0])]

'true'

# 9. Save Weights

In [31]:
model.save('action.h5')

In [32]:
del model

In [33]:
model.load_weights('action.h5')

NameError: name 'model' is not defined

# 10. Evaluation using Confusion Matrix and Accuracy

In [34]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [35]:
yhat = model.predict(X_test)

NameError: name 'model' is not defined

In [36]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

NameError: name 'yhat' is not defined

In [37]:
ytrue

[0]

In [38]:
multilabel_confusion_matrix(ytrue, yhat)

NameError: name 'yhat' is not defined

In [39]:
multilabel_confusion_matrix??

In [40]:
accuracy_score(ytrue, yhat)

NameError: name 'yhat' is not defined

# 11. Test in Real Time

In [41]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [42]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, face_mesh)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>


AttributeError: type object 'SolutionOutputs' has no attribute 'landmark'

In [ ]:
mp_face_mesh.FaceMesh??

In [43]:
cap.release()
cv2.destroyAllWindows()